<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in New York City</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import math

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

<a id='item1'></a>

## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [13]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [14]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [15]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [74]:
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ...
CLIENT_SECRET:...


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [17]:
neighborhoods.loc[0, 'Neighborhood']

'Wakefield'

Get the neighborhood's latitude and longitude values.

In [18]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [26]:
# type your answer here
LIMIT = 200
radius = 2000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=MBWHGPIUP3QCRNDQOIOOYL2Q0USBPXS4LM3MMLYOPOFGP2M5&client_secret=BWBX2CHRQCKGDYZ04ODBWGOST25WHN1MX3F2QGQQADIHIZW1&v=20180605&ll=40.89470517661,-73.84720052054902&radius=2000&limit=200&section=food'

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebbb7f0dd0f85001beb444f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wakefield',
  'headerFullLocation': 'Wakefield, Bronx',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 94,
  'suggestedBounds': {'ne': {'lat': 40.91270519461002,
    'lng': -73.82343272345271},
   'sw': {'lat': 40.87670515860999, 'lng': -73.87096831764534}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d375ce799fe8eec99fd2355',
       'name': 'Ripe Kitchen & Bar',
       'location': {'address': '151 W Sandford Blvd',
        'crossStreet': 'South 11th Avenue',
        'lat': 40.8981516856835

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875
1,Ali's Roti Shop,Caribbean Restaurant,40.894036,-73.856935
2,Jimbo's,Burger Joint,40.891740,-73.858226
3,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310
4,Kingston Tropical Bakery,Bakery,40.888568,-73.859885


And how many venues were returned by Foursquare?

In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

94 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            200)
            
        # make the GET request
        print('groups' in requests.get(url).json()["response"])
        if 'groups' in requests.get(url).json()["response"]:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        else:
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [32]:
# type your answer here

ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Wakefield
True
Co-op City
True
Eastchester
True
Fieldston
True
Riverdale
True
Kingsbridge
True
Marble Hill
True
Woodlawn
True
Norwood
True
Williamsbridge
True
Baychester
True
Pelham Parkway
True
City Island
True
Bedford Park
True
University Heights
True
Morris Heights
True
Fordham
True
East Tremont
True
West Farms
True
High  Bridge
True
Melrose
True
Mott Haven
True
Port Morris
True
Longwood
True
Hunts Point
True
Morrisania
True
Soundview
True
Clason Point
True
Throgs Neck
True
Country Club
True
Parkchester
True
Westchester Square
True
Van Nest
True
Morris Park
True
Belmont
True
Spuyten Duyvil
True
North Riverdale
True
Pelham Bay
True
Schuylerville
True
Edgewater Park
True
Castle Hill
True
Olinville
True
Pelham Gardens
True
Concourse
True
Unionport
True
Edenwald
True
Bay Ridge
True
Bensonhurst
True
Sunset Park
True
Greenpoint
True
Gravesend
True
Brighton Beach
True
Sheepshead Bay
True
Manhattan Terrace
True
Flatbush
True
Crown Heights
True
East Flatbush
True
Kensington
True
Windsor Terr

Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [33]:
print(ny_venues.shape)
ny_venues.head()

(26260, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
1,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Jimbo's,40.891740,-73.858226,Burger Joint
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Kingston Tropical Bakery,40.888568,-73.859885,Bakery


Let's check how many venues were returned for each neighborhood

In [34]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,100,100,100,100,100,100
Annadale,58,58,58,58,58,58
Arden Heights,76,76,76,76,76,76
Arlington,37,37,37,37,37,37
Arrochar,57,57,57,57,57,57
Arverne,31,31,31,31,31,31
Astoria,100,100,100,100,100,100
Astoria Heights,100,100,100,100,100,100
Auburndale,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [35]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 133 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [36]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Kofte Place,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint,Yemeni Restaurant
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [37]:
ny_onehot.shape

(26260, 134)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [38]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dosa Place,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Kofte Place,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint,Yemeni Restaurant
0,Allerton,0.0,0.0,0.000000,0.01,0.0,0.000000,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.020000,0.0,0.020000,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.09,0.0,0.040000,0.0,0.000000,0.0,0.0,0.0,0.060000,0.0,0.020000,0.040000,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.060000,0.000000,0.0,0.020000,0.0,0.0,0.020000,0.0,0.04,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.020000,0.000000,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.240000,0.0,0.000000,0.0,0.0,0.010000,0.0,0.0,0.0,0.120000,0.0,0.01,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.0,0.020000,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,Annadale,0.0,0.0,0.051724,0.00,0.0,0.017241,0.0,0.0,0.0,0.068966,0.051724,0.0,0.0,0.017241,0.0,0.017241,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.034483,0.0,0.017241,0.0,0.0,0.0,0.086207,0.0,0.051724,0.034483,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.034483,0.000000,0.0,0.017241,0.0,0.0,0.000000,0.0,0.00,0.017241,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068966,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.224138,0.0,0.000000,0.0,0.0,0.034483,0.0,0.0,0.0,0.051724,0.0,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.086207,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,Arden Heights,0.0,0.0,0.052632,0.00,0.0,0.052632,0.0,0.0,0.0,0.039474,0.039474,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0

#### Let's confirm the new size

In [39]:
ny_grouped.shape

(302, 134)

#### Let's print each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5
i = 0
for hood in ny_grouped['Neighborhood']:
    if i==5:
        break;
    i=i+1
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                  venue  freq
0           Pizza Place  0.24
1        Sandwich Place  0.12
2  Caribbean Restaurant  0.09
3         Deli / Bodega  0.06
4  Fast Food Restaurant  0.06


----Annadale----
                venue  freq
0         Pizza Place  0.22
1    Sushi Restaurant  0.09
2       Deli / Bodega  0.09
3  Italian Restaurant  0.07
4          Bagel Shop  0.07


----Arden Heights----
                venue  freq
0         Pizza Place  0.20
1  Italian Restaurant  0.09
2       Deli / Bodega  0.08
3    Sushi Restaurant  0.07
4  Chinese Restaurant  0.05


----Arlington----
                  venue  freq
0         Deli / Bodega  0.19
1  Fast Food Restaurant  0.14
2    Spanish Restaurant  0.08
3    Italian Restaurant  0.05
4           Wings Joint  0.05


----Arrochar----
                venue  freq
0  Italian Restaurant  0.19
1         Pizza Place  0.12
2       Deli / Bodega  0.12
3          Bagel Shop  0.05
4  Chinese Restaurant  0.05




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Sandwich Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Italian Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Chinese Restaurant
1,Annadale,Pizza Place,Deli / Bodega,Sushi Restaurant,Italian Restaurant,Bagel Shop,American Restaurant,Diner,Bakery,Sandwich Place,Fast Food Restaurant
2,Arden Heights,Pizza Place,Italian Restaurant,Deli / Bodega,Sushi Restaurant,Chinese Restaurant,American Restaurant,Donut Shop,Asian Restaurant,Food,Sandwich Place
3,Arlington,Deli / Bodega,Fast Food Restaurant,Spanish Restaurant,Donut Shop,Italian Restaurant,Wings Joint,Chinese Restaurant,Bagel Shop,Food Truck,Sushi Restaurant
4,Arrochar,Italian Restaurant,Pizza Place,Deli / Bodega,Bagel Shop,Bakery,Chinese Restaurant,Restaurant,Breakfast Spot,Food Truck,Steakhouse


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [43]:
ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)


In [44]:
for col in ny_grouped_clustering.columns: 
    print(col) 

Afghan Restaurant
African Restaurant
American Restaurant
Arepa Restaurant
Argentinian Restaurant
Asian Restaurant
Australian Restaurant
Austrian Restaurant
BBQ Joint
Bagel Shop
Bakery
Bistro
Brazilian Restaurant
Breakfast Spot
Buffet
Burger Joint
Burmese Restaurant
Burrito Place
Cafeteria
Café
Cajun / Creole Restaurant
Cambodian Restaurant
Cantonese Restaurant
Caribbean Restaurant
Caucasian Restaurant
Chinese Restaurant
Colombian Restaurant
Comfort Food Restaurant
Creperie
Cuban Restaurant
Czech Restaurant
Deli / Bodega
Dim Sum Restaurant
Diner
Donut Shop
Dosa Place
Dumpling Restaurant
Eastern European Restaurant
Egyptian Restaurant
Empanada Restaurant
English Restaurant
Ethiopian Restaurant
Falafel Restaurant
Fast Food Restaurant
Filipino Restaurant
Fish & Chips Shop
Food
Food Court
Food Stand
Food Truck
French Restaurant
Fried Chicken Joint
Gastropub
German Restaurant
Gluten-free Restaurant
Greek Restaurant
Halal Restaurant
Hawaiian Restaurant
Himalayan Restaurant
Hot Dog Joint
Hotpo

## I have analyzed the unique venue categories in NewYork. And listed the ones which are related to Indian style cuisines. They are 'Indian Restaurant', 'Dosa Place' and 'North Indian Restaurant'. Based on this information I am planning to do 3 types of clusters. And compare them to confirm the result from each clustering method is right. 

In [45]:
ny_allIndianTypes_clustering = ny_grouped_clustering[['Indian Restaurant', 'Dosa Place', 'North Indian Restaurant', 'South Indian Restaurant']]
ny_allIndianTypes_clustering.head()

,Indian Restaurant,Dosa Place,North Indian Restaurant,South Indian Restaurant
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [46]:
#ny_alltypessum_cluster.drop('Indian Restaurant', 1)
ny_grouped_clustering['AllSum'] = ny_grouped_clustering['Indian Restaurant'] + ny_grouped_clustering['North Indian Restaurant'] + ny_grouped_clustering['South Indian Restaurant'] + ny_grouped_clustering['Dosa Place']
ny_alltypessum_cluster = ny_grouped_clustering[['AllSum']]
ny_alltypessum_cluster.head()

,AllSum
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [47]:
ny_indianrest_clustering_3 = ny_grouped_clustering[['Indian Restaurant']]
ny_indianrest_clustering_3.head()

,Indian Restaurant
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [48]:
ny_indianrest_clustering_5 = ny_grouped_clustering[['Indian Restaurant']]
ny_indianrest_clustering_5.head()

,Indian Restaurant
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [49]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans_allTypesIndRest = KMeans(n_clusters=5, random_state=0).fit(ny_allIndianTypes_clustering)
kmeans_allIndRest3 = KMeans(n_clusters=3, random_state=0).fit(ny_indianrest_clustering_3)
kmeans_allIndRest5 = KMeans(n_clusters=5, random_state=0).fit(ny_indianrest_clustering_5)
kmeans_allrestSum = KMeans(n_clusters=5, random_state=0).fit(ny_alltypessum_cluster)

# check cluster labels generated for each row in the dataframe
print(kmeans_allTypesIndRest.labels_[0:50])
print(kmeans_allIndRest3.labels_[0:50])
print(kmeans_allIndRest5.labels_[0:50])
print(kmeans_allrestSum.labels_[0:50])

[0 0 0 0 0 0 3 3 3 0 3 3 3 0 1 0 0 3 0 4 0 2 0 0 0 3 0 0 3 0 4 3 0 1 0 0 0
 0 0 0 0 0 3 0 3 1 3 3 0 3]
[1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 2 1 2 1 1 1 0 1 1 1 1 2 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 0]
[0 0 0 0 0 0 2 2 2 0 2 2 2 0 3 0 0 2 0 1 0 4 0 0 0 2 0 0 2 0 1 2 0 3 0 0 0
 0 0 0 0 0 2 0 2 3 2 2 0 2]
[0 0 0 0 0 0 3 3 3 0 3 3 3 0 1 0 0 3 0 4 0 2 0 0 0 3 0 0 3 0 4 3 0 1 0 0 0
 0 0 0 0 0 3 0 3 1 3 3 0 3]


In [50]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Sandwich Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Italian Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Chinese Restaurant
1,Annadale,Pizza Place,Deli / Bodega,Sushi Restaurant,Italian Restaurant,Bagel Shop,American Restaurant,Diner,Bakery,Sandwich Place,Fast Food Restaurant
2,Arden Heights,Pizza Place,Italian Restaurant,Deli / Bodega,Sushi Restaurant,Chinese Restaurant,American Restaurant,Donut Shop,Asian Restaurant,Food,Sandwich Place
3,Arlington,Deli / Bodega,Fast Food Restaurant,Spanish Restaurant,Donut Shop,Italian Restaurant,Wings Joint,Chinese Restaurant,Bagel Shop,Food Truck,Sushi Restaurant
4,Arrochar,Italian Restaurant,Pizza Place,Deli / Bodega,Bagel Shop,Bakery,Chinese Restaurant,Restaurant,Breakfast Spot,Food Truck,Steakhouse


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_AllTypes', kmeans_allTypesIndRest.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster_AllIndRes3', kmeans_allIndRest3.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster_AllIndRes5', kmeans_allIndRest5.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster_AllSum', kmeans_allrestSum.labels_)

ny_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns!
#ny_merged['Cluster Labels']

,Borough,Neighborhood,Latitude,Longitude,Cluster_AllSum,Cluster_AllIndRes5,Cluster_AllIndRes3,Cluster_AllTypes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,3,2,1,3,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant,Bakery,Fried Chicken Joint,Italian Restaurant,American Restaurant,Donut Shop
1,Bronx,Co-op City,40.874294,-73.829939,0,0,1,0,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Bakery,Fried Chicken Joint,Spanish Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,0,0,1,0,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Chinese Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Fried Chicken Joint,Diner,Burger Joint
3,Bronx,Fieldston,40.895437,-73.905643,3,2,1,3,Pizza Place,Mexican Restaurant,Sandwich Place,Deli / Bodega,Donut Shop,Diner,Latin American Restaurant,Bakery,Burger Joint,Chinese Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,3,2,1,3,Pizza Place,Deli / Bodega,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Burger Joint,Latin American Restaurant,Italian Restaurant,Bakery


Finally, let's visualize the resulting clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
rainbow = ['#8000ff', '#FE0303', '#049A41', '#790272', '#EEFE03']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster_AllTypes']):
    if math.isnan(cluster):
        continue

    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [53]:
##### create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
rainbow = ['#8000ff', '#FE0303', '#049A41', '#790272', '#EEFE03']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster_AllIndRes3']):
    if math.isnan(cluster):
        continue

    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2.5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
rainbow = ['#8000ff', '#FE0303', '#049A41', '#790272', '#EEFE03']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster_AllIndRes5']):
    if math.isnan(cluster):
        continue

    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2.5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
rainbow = ['#8000ff', '#FE0303', '#049A41', '#790272', '#EEFE03']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster_AllSum']):
    if math.isnan(cluster):
        continue

    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2.5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1/3 Indian Restaurant 3 Clusters

In [56]:
ny_merged.loc[ny_merged['Cluster_AllIndRes3'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Woodlawn,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Donut Shop,American Restaurant,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Diner,Bakery
26,Soundview,Bronx,Pizza Place,Fast Food Restaurant,Spanish Restaurant,Deli / Bodega,Fried Chicken Joint,Latin American Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,American Restaurant
40,Castle Hill,Bronx,Pizza Place,Chinese Restaurant,Diner,Fried Chicken Joint,Deli / Bodega,Sandwich Place,Breakfast Spot,Fast Food Restaurant,American Restaurant,Donut Shop
46,Bay Ridge,Brooklyn,Pizza Place,Italian Restaurant,Bakery,Bagel Shop,Mexican Restaurant,Greek Restaurant,Japanese Restaurant,Café,Sushi Restaurant,Chinese Restaurant
54,Flatbush,Brooklyn,Caribbean Restaurant,Pizza Place,Mexican Restaurant,Café,Bakery,Deli / Bodega,Bagel Shop,Thai Restaurant,Latin American Restaurant,Chinese Restaurant
55,Crown Heights,Brooklyn,Caribbean Restaurant,Café,Pizza Place,Bakery,Southern / Soul Food Restaurant,Mexican Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,American Restaurant
56,East Flatbush,Brooklyn,Caribbean Restaurant,Pizza Place,Sandwich Place,Chinese Restaurant,Fried Chicken Joint,Donut Shop,Fast Food Restaurant,Deli / Bodega,Diner,Mexican Restaurant
57,Kensington,Brooklyn,Pizza Place,Café,Mexican Restaurant,Diner,Middle Eastern Restaurant,Restaurant,Deli / Bodega,Thai Restaurant,Bagel Shop,Italian Restaurant
59,Prospect Heights,Brooklyn,Bakery,Café,American Restaurant,Pizza Place,Thai Restaurant,Mexican Restaurant,New American Restaurant,Italian Restaurant,Sushi Restaurant,Bagel Shop
63,Bedford Stuyvesant,Brooklyn,Café,Bakery,Caribbean Restaurant,Mexican Restaurant,Pizza Place,Southern / Soul Food Restaurant,New American Restaurant,Deli / Bodega,American Restaurant,African Restaurant


#### Cluster 2/3 Indian Restaurant 3 Clusters

In [57]:
ny_merged.loc[ny_merged['Cluster_AllIndRes3'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant,Bakery,Fried Chicken Joint,Italian Restaurant,American Restaurant,Donut Shop
1,Co-op City,Bronx,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Bakery,Fried Chicken Joint,Spanish Restaurant
2,Eastchester,Bronx,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Chinese Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Fried Chicken Joint,Diner,Burger Joint
3,Fieldston,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Deli / Bodega,Donut Shop,Diner,Latin American Restaurant,Bakery,Burger Joint,Chinese Restaurant
4,Riverdale,Bronx,Pizza Place,Deli / Bodega,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Burger Joint,Latin American Restaurant,Italian Restaurant,Bakery
5,Kingsbridge,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Deli / Bodega,Spanish Restaurant,Bakery,Chinese Restaurant,Fast Food Restaurant
6,Marble Hill,Manhattan,Pizza Place,Mexican Restaurant,Diner,Latin American Restaurant,Café,Sandwich Place,Chinese Restaurant,Bakery,Donut Shop,Deli / Bodega
8,Norwood,Bronx,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Chinese Restaurant,Mexican Restaurant,Diner,Fast Food Restaurant,Deli / Bodega,Spanish Restaurant
9,Williamsbridge,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Chinese Restaurant,Seafood Restaurant,Restaurant
10,Baychester,Bronx,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Donut Shop,Bakery,Deli / Bodega,Italian Restaurant,Asian Restaurant


#### Cluster 3/3 Indian Restaurant 3 Clusters

In [58]:
ny_merged.loc[ny_merged['Cluster_AllIndRes3'] == 2, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
130,Woodside,Queens,Thai Restaurant,Mexican Restaurant,Bakery,Indian Restaurant,Food Truck,Pizza Place,Latin American Restaurant,American Restaurant,Filipino Restaurant,Italian Restaurant
131,Jackson Heights,Queens,Thai Restaurant,Bakery,Indian Restaurant,Latin American Restaurant,Food Truck,Argentinian Restaurant,Pizza Place,Mexican Restaurant,South American Restaurant,Chinese Restaurant
132,Elmhurst,Queens,Thai Restaurant,Bakery,Food Truck,Indian Restaurant,Argentinian Restaurant,Mexican Restaurant,Latin American Restaurant,Filipino Restaurant,South American Restaurant,Indonesian Restaurant
136,Kew Gardens,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,New American Restaurant,Diner
137,Richmond Hill,Queens,Pizza Place,Sandwich Place,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Deli / Bodega,Latin American Restaurant,Diner,Fast Food Restaurant,Bakery
155,Glen Oaks,Queens,Indian Restaurant,Pizza Place,Chinese Restaurant,Food Truck,Diner,Fast Food Restaurant,Café,Deli / Bodega,Bakery,Sandwich Place
156,Bellerose,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Bagel Shop,Food Truck,Sandwich Place,Bakery
159,Briarwood,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Fried Chicken Joint,Chinese Restaurant,Donut Shop,Fast Food Restaurant,Latin American Restaurant,Sushi Restaurant,Portuguese Restaurant
181,Floral Park,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Diner,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Bakery,Restaurant
194,Bellaire,Queens,Pizza Place,Chinese Restaurant,Deli / Bodega,Italian Restaurant,Fast Food Restaurant,Indian Restaurant,Donut Shop,Sushi Restaurant,Fried Chicken Joint,Latin American Restaurant


#### Cluster 1/5 Indian Restaurant 5 Clusters

In [59]:
ny_merged.loc[ny_merged['Cluster_AllIndRes5'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bronx,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Bakery,Fried Chicken Joint,Spanish Restaurant
2,Eastchester,Bronx,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Chinese Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Fried Chicken Joint,Diner,Burger Joint
6,Marble Hill,Manhattan,Pizza Place,Mexican Restaurant,Diner,Latin American Restaurant,Café,Sandwich Place,Chinese Restaurant,Bakery,Donut Shop,Deli / Bodega
8,Norwood,Bronx,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Chinese Restaurant,Mexican Restaurant,Diner,Fast Food Restaurant,Deli / Bodega,Spanish Restaurant
9,Williamsbridge,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Chinese Restaurant,Seafood Restaurant,Restaurant
10,Baychester,Bronx,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Donut Shop,Bakery,Deli / Bodega,Italian Restaurant,Asian Restaurant
11,Pelham Parkway,Bronx,Pizza Place,Sandwich Place,Deli / Bodega,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Caribbean Restaurant,Donut Shop,Fast Food Restaurant,Bakery
12,City Island,Bronx,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Café,Donut Shop,Sandwich Place,Spanish Restaurant,Chinese Restaurant,French Restaurant,Tapas Restaurant
13,Bedford Park,Bronx,Italian Restaurant,Pizza Place,Diner,Deli / Bodega,Mexican Restaurant,Sandwich Place,Caribbean Restaurant,Spanish Restaurant,Latin American Restaurant,Bakery
14,University Heights,Bronx,Pizza Place,Italian Restaurant,Latin American Restaurant,Mexican Restaurant,Deli / Bodega,Spanish Restaurant,Café,Bakery,Diner,Restaurant


#### Cluster 2/5 Indian Restaurant 5 Clusters

In [60]:
ny_merged.loc[ny_merged['Cluster_AllIndRes5'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
130,Woodside,Queens,Thai Restaurant,Mexican Restaurant,Bakery,Indian Restaurant,Food Truck,Pizza Place,Latin American Restaurant,American Restaurant,Filipino Restaurant,Italian Restaurant
131,Jackson Heights,Queens,Thai Restaurant,Bakery,Indian Restaurant,Latin American Restaurant,Food Truck,Argentinian Restaurant,Pizza Place,Mexican Restaurant,South American Restaurant,Chinese Restaurant
132,Elmhurst,Queens,Thai Restaurant,Bakery,Food Truck,Indian Restaurant,Argentinian Restaurant,Mexican Restaurant,Latin American Restaurant,Filipino Restaurant,South American Restaurant,Indonesian Restaurant
136,Kew Gardens,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,New American Restaurant,Diner
137,Richmond Hill,Queens,Pizza Place,Sandwich Place,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Deli / Bodega,Latin American Restaurant,Diner,Fast Food Restaurant,Bakery
159,Briarwood,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Fried Chicken Joint,Chinese Restaurant,Donut Shop,Fast Food Restaurant,Latin American Restaurant,Sushi Restaurant,Portuguese Restaurant
194,Bellaire,Queens,Pizza Place,Chinese Restaurant,Deli / Bodega,Italian Restaurant,Fast Food Restaurant,Indian Restaurant,Donut Shop,Sushi Restaurant,Fried Chicken Joint,Latin American Restaurant
274,Tudor City,Manhattan,Indian Restaurant,American Restaurant,Japanese Restaurant,Mediterranean Restaurant,Korean Restaurant,Sandwich Place,Steakhouse,Pizza Place,Café,Italian Restaurant


#### Cluster 3/5 Indian Restaurant 5 Clusters

In [61]:
ny_merged.loc[ny_merged['Cluster_AllIndRes5'] == 2, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant,Bakery,Fried Chicken Joint,Italian Restaurant,American Restaurant,Donut Shop
3,Fieldston,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Deli / Bodega,Donut Shop,Diner,Latin American Restaurant,Bakery,Burger Joint,Chinese Restaurant
4,Riverdale,Bronx,Pizza Place,Deli / Bodega,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Burger Joint,Latin American Restaurant,Italian Restaurant,Bakery
5,Kingsbridge,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Deli / Bodega,Spanish Restaurant,Bakery,Chinese Restaurant,Fast Food Restaurant
7,Woodlawn,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Donut Shop,American Restaurant,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Diner,Bakery
15,Morris Heights,Bronx,Pizza Place,Latin American Restaurant,Spanish Restaurant,Deli / Bodega,Bakery,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Tapas Restaurant,Seafood Restaurant
20,Melrose,Bronx,Pizza Place,Sandwich Place,Mexican Restaurant,Fast Food Restaurant,Donut Shop,Diner,Fried Chicken Joint,Caribbean Restaurant,Deli / Bodega,Latin American Restaurant
25,Morrisania,Bronx,Pizza Place,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Donut Shop,Fried Chicken Joint,Diner,Spanish Restaurant,Deli / Bodega,Latin American Restaurant
26,Soundview,Bronx,Pizza Place,Fast Food Restaurant,Spanish Restaurant,Deli / Bodega,Fried Chicken Joint,Latin American Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,American Restaurant
30,Parkchester,Bronx,Pizza Place,Sandwich Place,Latin American Restaurant,Spanish Restaurant,Donut Shop,Diner,Italian Restaurant,Deli / Bodega,Breakfast Spot,American Restaurant


#### Cluster 4/5 Indian Restaurant 5 Clusters

In [62]:
ny_merged.loc[ny_merged['Cluster_AllIndRes5'] == 3, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Castle Hill,Bronx,Pizza Place,Chinese Restaurant,Diner,Fried Chicken Joint,Deli / Bodega,Sandwich Place,Breakfast Spot,Fast Food Restaurant,American Restaurant,Donut Shop
54,Flatbush,Brooklyn,Caribbean Restaurant,Pizza Place,Mexican Restaurant,Café,Bakery,Deli / Bodega,Bagel Shop,Thai Restaurant,Latin American Restaurant,Chinese Restaurant
67,Red Hook,Brooklyn,Food Truck,Seafood Restaurant,Pizza Place,Italian Restaurant,American Restaurant,Deli / Bodega,Café,Bakery,Thai Restaurant,Mexican Restaurant
89,Ocean Hill,Brooklyn,Pizza Place,Deli / Bodega,Café,Bakery,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,Southern / Soul Food Restaurant,Donut Shop,Fried Chicken Joint
93,Prospect Park South,Brooklyn,Caribbean Restaurant,Café,Mexican Restaurant,Pizza Place,Thai Restaurant,Bakery,Middle Eastern Restaurant,Deli / Bodega,Italian Restaurant,Indian Restaurant
111,Upper West Side,Manhattan,Bakery,Italian Restaurant,American Restaurant,Pizza Place,Chinese Restaurant,Café,Mediterranean Restaurant,Sushi Restaurant,Bagel Shop,Mexican Restaurant
115,Murray Hill,Manhattan,Korean Restaurant,Chinese Restaurant,Pizza Place,American Restaurant,Japanese Restaurant,Indian Restaurant,Bakery,Dumpling Restaurant,Hotpot Restaurant,Mediterranean Restaurant
124,Manhattan Valley,Manhattan,Bakery,American Restaurant,Italian Restaurant,Pizza Place,Café,Chinese Restaurant,Bagel Shop,Indian Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant
125,Morningside Heights,Manhattan,Italian Restaurant,American Restaurant,Seafood Restaurant,Pizza Place,Bakery,Café,Mexican Restaurant,Chinese Restaurant,French Restaurant,Indian Restaurant
126,Gramercy,Manhattan,American Restaurant,Pizza Place,Japanese Restaurant,New American Restaurant,Café,Korean Restaurant,Bagel Shop,Indian Restaurant,Italian Restaurant,Mediterranean Restaurant


#### Cluster 5/5 Indian Restaurant 5 Clusters

In [63]:
ny_merged.loc[ny_merged['Cluster_AllIndRes5'] == 4, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
155,Glen Oaks,Queens,Indian Restaurant,Pizza Place,Chinese Restaurant,Food Truck,Diner,Fast Food Restaurant,Café,Deli / Bodega,Bakery,Sandwich Place
156,Bellerose,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Bagel Shop,Food Truck,Sandwich Place,Bakery
181,Floral Park,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Diner,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Bakery,Restaurant


#### Cluster 1/5 AllTypes of Indian Restaurant 5 Clusters

In [64]:
ny_merged.loc[ny_merged['Cluster_AllTypes'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bronx,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Bakery,Fried Chicken Joint,Spanish Restaurant
2,Eastchester,Bronx,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Chinese Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Fried Chicken Joint,Diner,Burger Joint
6,Marble Hill,Manhattan,Pizza Place,Mexican Restaurant,Diner,Latin American Restaurant,Café,Sandwich Place,Chinese Restaurant,Bakery,Donut Shop,Deli / Bodega
8,Norwood,Bronx,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Chinese Restaurant,Mexican Restaurant,Diner,Fast Food Restaurant,Deli / Bodega,Spanish Restaurant
9,Williamsbridge,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Chinese Restaurant,Seafood Restaurant,Restaurant
10,Baychester,Bronx,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Donut Shop,Bakery,Deli / Bodega,Italian Restaurant,Asian Restaurant
11,Pelham Parkway,Bronx,Pizza Place,Sandwich Place,Deli / Bodega,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Caribbean Restaurant,Donut Shop,Fast Food Restaurant,Bakery
12,City Island,Bronx,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Café,Donut Shop,Sandwich Place,Spanish Restaurant,Chinese Restaurant,French Restaurant,Tapas Restaurant
13,Bedford Park,Bronx,Italian Restaurant,Pizza Place,Diner,Deli / Bodega,Mexican Restaurant,Sandwich Place,Caribbean Restaurant,Spanish Restaurant,Latin American Restaurant,Bakery
14,University Heights,Bronx,Pizza Place,Italian Restaurant,Latin American Restaurant,Mexican Restaurant,Deli / Bodega,Spanish Restaurant,Café,Bakery,Diner,Restaurant


#### Cluster 2/5 AllTypes of Indian Restaurant 5 Clusters

In [65]:
ny_merged.loc[ny_merged['Cluster_AllTypes'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Castle Hill,Bronx,Pizza Place,Chinese Restaurant,Diner,Fried Chicken Joint,Deli / Bodega,Sandwich Place,Breakfast Spot,Fast Food Restaurant,American Restaurant,Donut Shop
54,Flatbush,Brooklyn,Caribbean Restaurant,Pizza Place,Mexican Restaurant,Café,Bakery,Deli / Bodega,Bagel Shop,Thai Restaurant,Latin American Restaurant,Chinese Restaurant
67,Red Hook,Brooklyn,Food Truck,Seafood Restaurant,Pizza Place,Italian Restaurant,American Restaurant,Deli / Bodega,Café,Bakery,Thai Restaurant,Mexican Restaurant
89,Ocean Hill,Brooklyn,Pizza Place,Deli / Bodega,Café,Bakery,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,Southern / Soul Food Restaurant,Donut Shop,Fried Chicken Joint
93,Prospect Park South,Brooklyn,Caribbean Restaurant,Café,Mexican Restaurant,Pizza Place,Thai Restaurant,Bakery,Middle Eastern Restaurant,Deli / Bodega,Italian Restaurant,Indian Restaurant
111,Upper West Side,Manhattan,Bakery,Italian Restaurant,American Restaurant,Pizza Place,Chinese Restaurant,Café,Mediterranean Restaurant,Sushi Restaurant,Bagel Shop,Mexican Restaurant
115,Murray Hill,Manhattan,Korean Restaurant,Chinese Restaurant,Pizza Place,American Restaurant,Japanese Restaurant,Indian Restaurant,Bakery,Dumpling Restaurant,Hotpot Restaurant,Mediterranean Restaurant
124,Manhattan Valley,Manhattan,Bakery,American Restaurant,Italian Restaurant,Pizza Place,Café,Chinese Restaurant,Bagel Shop,Indian Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant
125,Morningside Heights,Manhattan,Italian Restaurant,American Restaurant,Seafood Restaurant,Pizza Place,Bakery,Café,Mexican Restaurant,Chinese Restaurant,French Restaurant,Indian Restaurant
126,Gramercy,Manhattan,American Restaurant,Pizza Place,Japanese Restaurant,New American Restaurant,Café,Korean Restaurant,Bagel Shop,Indian Restaurant,Italian Restaurant,Mediterranean Restaurant


#### Cluster 3/5 AllTypes of Indian Restaurant 5 Clusters

In [66]:
ny_merged.loc[ny_merged['Cluster_AllTypes'] == 2, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
155,Glen Oaks,Queens,Indian Restaurant,Pizza Place,Chinese Restaurant,Food Truck,Diner,Fast Food Restaurant,Café,Deli / Bodega,Bakery,Sandwich Place
156,Bellerose,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Bagel Shop,Food Truck,Sandwich Place,Bakery
181,Floral Park,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Diner,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Bakery,Restaurant


#### Cluster 4/5 AllTypes of Indian Restaurant 5 Clusters

In [67]:
ny_merged.loc[ny_merged['Cluster_AllTypes'] == 3, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant,Bakery,Fried Chicken Joint,Italian Restaurant,American Restaurant,Donut Shop
3,Fieldston,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Deli / Bodega,Donut Shop,Diner,Latin American Restaurant,Bakery,Burger Joint,Chinese Restaurant
4,Riverdale,Bronx,Pizza Place,Deli / Bodega,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Burger Joint,Latin American Restaurant,Italian Restaurant,Bakery
5,Kingsbridge,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Deli / Bodega,Spanish Restaurant,Bakery,Chinese Restaurant,Fast Food Restaurant
7,Woodlawn,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Donut Shop,American Restaurant,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Diner,Bakery
15,Morris Heights,Bronx,Pizza Place,Latin American Restaurant,Spanish Restaurant,Deli / Bodega,Bakery,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Tapas Restaurant,Seafood Restaurant
20,Melrose,Bronx,Pizza Place,Sandwich Place,Mexican Restaurant,Fast Food Restaurant,Donut Shop,Diner,Fried Chicken Joint,Caribbean Restaurant,Deli / Bodega,Latin American Restaurant
25,Morrisania,Bronx,Pizza Place,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Donut Shop,Fried Chicken Joint,Diner,Spanish Restaurant,Deli / Bodega,Latin American Restaurant
26,Soundview,Bronx,Pizza Place,Fast Food Restaurant,Spanish Restaurant,Deli / Bodega,Fried Chicken Joint,Latin American Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,American Restaurant
30,Parkchester,Bronx,Pizza Place,Sandwich Place,Latin American Restaurant,Spanish Restaurant,Donut Shop,Diner,Italian Restaurant,Deli / Bodega,Breakfast Spot,American Restaurant


#### Cluster 5/5 AllTypes of Indian Restaurant 5 Clusters

In [68]:
ny_merged.loc[ny_merged['Cluster_AllTypes'] == 4, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
130,Woodside,Queens,Thai Restaurant,Mexican Restaurant,Bakery,Indian Restaurant,Food Truck,Pizza Place,Latin American Restaurant,American Restaurant,Filipino Restaurant,Italian Restaurant
131,Jackson Heights,Queens,Thai Restaurant,Bakery,Indian Restaurant,Latin American Restaurant,Food Truck,Argentinian Restaurant,Pizza Place,Mexican Restaurant,South American Restaurant,Chinese Restaurant
132,Elmhurst,Queens,Thai Restaurant,Bakery,Food Truck,Indian Restaurant,Argentinian Restaurant,Mexican Restaurant,Latin American Restaurant,Filipino Restaurant,South American Restaurant,Indonesian Restaurant
136,Kew Gardens,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,New American Restaurant,Diner
137,Richmond Hill,Queens,Pizza Place,Sandwich Place,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Deli / Bodega,Latin American Restaurant,Diner,Fast Food Restaurant,Bakery
159,Briarwood,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Fried Chicken Joint,Chinese Restaurant,Donut Shop,Fast Food Restaurant,Latin American Restaurant,Sushi Restaurant,Portuguese Restaurant
194,Bellaire,Queens,Pizza Place,Chinese Restaurant,Deli / Bodega,Italian Restaurant,Fast Food Restaurant,Indian Restaurant,Donut Shop,Sushi Restaurant,Fried Chicken Joint,Latin American Restaurant
274,Tudor City,Manhattan,Indian Restaurant,American Restaurant,Japanese Restaurant,Mediterranean Restaurant,Korean Restaurant,Sandwich Place,Steakhouse,Pizza Place,Café,Italian Restaurant


#### Cluster 1/5 Sum of Means of AllTypes of Indian Restaurant 5 Clusters

In [69]:
ny_merged.loc[ny_merged['Cluster_AllSum'] == 0, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bronx,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Bakery,Fried Chicken Joint,Spanish Restaurant
2,Eastchester,Bronx,Fast Food Restaurant,Caribbean Restaurant,Pizza Place,Chinese Restaurant,Deli / Bodega,Sandwich Place,Donut Shop,Fried Chicken Joint,Diner,Burger Joint
6,Marble Hill,Manhattan,Pizza Place,Mexican Restaurant,Diner,Latin American Restaurant,Café,Sandwich Place,Chinese Restaurant,Bakery,Donut Shop,Deli / Bodega
8,Norwood,Bronx,Pizza Place,Sandwich Place,Caribbean Restaurant,Donut Shop,Chinese Restaurant,Mexican Restaurant,Diner,Fast Food Restaurant,Deli / Bodega,Spanish Restaurant
9,Williamsbridge,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Bakery,Chinese Restaurant,Seafood Restaurant,Restaurant
10,Baychester,Bronx,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Donut Shop,Bakery,Deli / Bodega,Italian Restaurant,Asian Restaurant
11,Pelham Parkway,Bronx,Pizza Place,Sandwich Place,Deli / Bodega,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Caribbean Restaurant,Donut Shop,Fast Food Restaurant,Bakery
12,City Island,Bronx,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Café,Donut Shop,Sandwich Place,Spanish Restaurant,Chinese Restaurant,French Restaurant,Tapas Restaurant
13,Bedford Park,Bronx,Italian Restaurant,Pizza Place,Diner,Deli / Bodega,Mexican Restaurant,Sandwich Place,Caribbean Restaurant,Spanish Restaurant,Latin American Restaurant,Bakery
14,University Heights,Bronx,Pizza Place,Italian Restaurant,Latin American Restaurant,Mexican Restaurant,Deli / Bodega,Spanish Restaurant,Café,Bakery,Diner,Restaurant


#### Cluster 2/5 Sum of Means of AllTypes of Indian Restaurant 5 Clusters

In [70]:
ny_merged.loc[ny_merged['Cluster_AllSum'] == 1, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Soundview,Bronx,Pizza Place,Fast Food Restaurant,Spanish Restaurant,Deli / Bodega,Fried Chicken Joint,Latin American Restaurant,Chinese Restaurant,Sandwich Place,Donut Shop,American Restaurant
40,Castle Hill,Bronx,Pizza Place,Chinese Restaurant,Diner,Fried Chicken Joint,Deli / Bodega,Sandwich Place,Breakfast Spot,Fast Food Restaurant,American Restaurant,Donut Shop
54,Flatbush,Brooklyn,Caribbean Restaurant,Pizza Place,Mexican Restaurant,Café,Bakery,Deli / Bodega,Bagel Shop,Thai Restaurant,Latin American Restaurant,Chinese Restaurant
67,Red Hook,Brooklyn,Food Truck,Seafood Restaurant,Pizza Place,Italian Restaurant,American Restaurant,Deli / Bodega,Café,Bakery,Thai Restaurant,Mexican Restaurant
89,Ocean Hill,Brooklyn,Pizza Place,Deli / Bodega,Café,Bakery,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,Southern / Soul Food Restaurant,Donut Shop,Fried Chicken Joint
93,Prospect Park South,Brooklyn,Caribbean Restaurant,Café,Mexican Restaurant,Pizza Place,Thai Restaurant,Bakery,Middle Eastern Restaurant,Deli / Bodega,Italian Restaurant,Indian Restaurant
111,Upper West Side,Manhattan,Bakery,Italian Restaurant,American Restaurant,Pizza Place,Chinese Restaurant,Café,Mediterranean Restaurant,Sushi Restaurant,Bagel Shop,Mexican Restaurant
114,Midtown,Manhattan,Steakhouse,American Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Japanese Restaurant,Korean Restaurant,Burger Joint,Sandwich Place,Bakery,Chinese Restaurant
115,Murray Hill,Manhattan,Korean Restaurant,Chinese Restaurant,Pizza Place,American Restaurant,Japanese Restaurant,Indian Restaurant,Bakery,Dumpling Restaurant,Hotpot Restaurant,Mediterranean Restaurant
124,Manhattan Valley,Manhattan,Bakery,American Restaurant,Italian Restaurant,Pizza Place,Café,Chinese Restaurant,Bagel Shop,Indian Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant


#### Cluster 3/5 Sum of Means of AllTypes of Indian Restaurant 5 Clusters

In [71]:
ny_merged.loc[ny_merged['Cluster_AllSum'] == 2, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
155,Glen Oaks,Queens,Indian Restaurant,Pizza Place,Chinese Restaurant,Food Truck,Diner,Fast Food Restaurant,Café,Deli / Bodega,Bakery,Sandwich Place
156,Bellerose,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Bagel Shop,Food Truck,Sandwich Place,Bakery
181,Floral Park,Queens,Indian Restaurant,Deli / Bodega,Pizza Place,Diner,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Bakery,Restaurant


#### Cluster 4/5 Sum of Means of AllTypes of Indian Restaurant 5 Clusters

In [72]:
ny_merged.loc[ny_merged['Cluster_AllSum'] == 3, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant,Bakery,Fried Chicken Joint,Italian Restaurant,American Restaurant,Donut Shop
3,Fieldston,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Deli / Bodega,Donut Shop,Diner,Latin American Restaurant,Bakery,Burger Joint,Chinese Restaurant
4,Riverdale,Bronx,Pizza Place,Deli / Bodega,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Burger Joint,Latin American Restaurant,Italian Restaurant,Bakery
5,Kingsbridge,Bronx,Pizza Place,Mexican Restaurant,Sandwich Place,Donut Shop,Diner,Deli / Bodega,Spanish Restaurant,Bakery,Chinese Restaurant,Fast Food Restaurant
7,Woodlawn,Bronx,Pizza Place,Caribbean Restaurant,Deli / Bodega,Donut Shop,American Restaurant,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Diner,Bakery
15,Morris Heights,Bronx,Pizza Place,Latin American Restaurant,Spanish Restaurant,Deli / Bodega,Bakery,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Tapas Restaurant,Seafood Restaurant
20,Melrose,Bronx,Pizza Place,Sandwich Place,Mexican Restaurant,Fast Food Restaurant,Donut Shop,Diner,Fried Chicken Joint,Caribbean Restaurant,Deli / Bodega,Latin American Restaurant
25,Morrisania,Bronx,Pizza Place,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Donut Shop,Fried Chicken Joint,Diner,Spanish Restaurant,Deli / Bodega,Latin American Restaurant
30,Parkchester,Bronx,Pizza Place,Sandwich Place,Latin American Restaurant,Spanish Restaurant,Donut Shop,Diner,Italian Restaurant,Deli / Bodega,Breakfast Spot,American Restaurant
43,Concourse,Bronx,Sandwich Place,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Deli / Bodega,Donut Shop,Spanish Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant


#### Cluster 5/5 Sum of Means of AllTypes of Indian Restaurant 5 Clusters

In [73]:
ny_merged.loc[ny_merged['Cluster_AllSum'] == 4, ny_merged.columns[[1] + [0] + list(range(8, ny_merged.shape[1]))]]

,Neighborhood,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
130,Woodside,Queens,Thai Restaurant,Mexican Restaurant,Bakery,Indian Restaurant,Food Truck,Pizza Place,Latin American Restaurant,American Restaurant,Filipino Restaurant,Italian Restaurant
131,Jackson Heights,Queens,Thai Restaurant,Bakery,Indian Restaurant,Latin American Restaurant,Food Truck,Argentinian Restaurant,Pizza Place,Mexican Restaurant,South American Restaurant,Chinese Restaurant
132,Elmhurst,Queens,Thai Restaurant,Bakery,Food Truck,Indian Restaurant,Argentinian Restaurant,Mexican Restaurant,Latin American Restaurant,Filipino Restaurant,South American Restaurant,Indonesian Restaurant
136,Kew Gardens,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,New American Restaurant,Diner
137,Richmond Hill,Queens,Pizza Place,Sandwich Place,Chinese Restaurant,Indian Restaurant,Caribbean Restaurant,Deli / Bodega,Latin American Restaurant,Diner,Fast Food Restaurant,Bakery
159,Briarwood,Queens,Pizza Place,Sandwich Place,Indian Restaurant,Fried Chicken Joint,Chinese Restaurant,Donut Shop,Fast Food Restaurant,Latin American Restaurant,Sushi Restaurant,Portuguese Restaurant
194,Bellaire,Queens,Pizza Place,Chinese Restaurant,Deli / Bodega,Italian Restaurant,Fast Food Restaurant,Indian Restaurant,Donut Shop,Sushi Restaurant,Fried Chicken Joint,Latin American Restaurant
274,Tudor City,Manhattan,Indian Restaurant,American Restaurant,Japanese Restaurant,Mediterranean Restaurant,Korean Restaurant,Sandwich Place,Steakhouse,Pizza Place,Café,Italian Restaurant


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).